# Results from MILP Realtime Computations

In [1]:
import numpy as np
import pandas as pd

from ctmmodels.const import *

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'experiments/'

In [3]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

# Varying parameters: demand, alpha, beta, gamma

## Saving dataframes

In [4]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Loading CSV files

In [5]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

_col_rename = {
    'Runtime': 'runtime',
    'Delay': 'delay',
    'Throughput': 'throughput',
    'ObjValue': 'objective_value'
}

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [6]:
_results_dflist = []

for demand in _demands:
    for weights in _weights:
        if weights == (0,0,0):
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_old.csv'.format(demand))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_old.csv'.format(demand))
            df_e3 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch3_old.csv'.format(demand))
        else:
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e3 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch3_a{}_b{}_c{}.csv'.format(demand, *weights))

        df_e1 = df_e1.rename(columns=_col_rename)
        df_e1['demand'] = [demand]
        df_e1['new_model'] = weights != (0,0,0)
        df_e1['alpha'] = weights[0]
        df_e1['beta'] = weights[1]
        df_e1['gamma'] = weights[2]
        df_e1['model_type'] = _model_type[weights]
        df_e1['epoch'] = 1
        
        df_e2 = df_e2.rename(columns=_col_rename)
        df_e2['demand'] = [demand]
        df_e2['new_model'] = weights != (0,0,0)
        df_e2['alpha'] = weights[0]
        df_e2['beta'] = weights[1]
        df_e2['gamma'] = weights[2]
        df_e2['model_type'] = _model_type[weights]
        df_e2['epoch'] = 2
        
        df_e3 = df_e3.rename(columns=_col_rename)
        df_e3['demand'] = [demand]
        df_e3['new_model'] = weights != (0,0,0)
        df_e3['alpha'] = weights[0]
        df_e3['beta'] = weights[1]
        df_e3['gamma'] = weights[2]
        df_e3['model_type'] = _model_type[weights]
        df_e3['epoch'] = 3
        
        _results_dflist.append(pd.concat([df_e1, df_e2, df_e3]))

In [7]:
df = pd.concat(_results_dflist)

## Adding initial MILP results

In [9]:
df_initial_results = pd.read_pickle(DF_PATH + 'results.pkl')

In [10]:
df_initial_results.head()

,demand_ns,demand_ew,runtime,delay,throughput,objective_value,alpha,beta,gamma
0,450,450,165.902929,1534.544304,51.278481,709.539027,1.0,0.0,0.0
1,450,450,268.785380,1750.000000,55.000000,-205.729167,0.0,1.0,0.0
2,450,450,573.475265,1632.632911,52.278481,-2055.801688,0.0,0.0,1.0
3,450,450,475.487772,1534.544304,51.278481,258.865157,0.5,0.5,0.0
4,450,450,293.788045,1610.215190,54.000000,-1128.895162,0.0,0.5,0.5


In [11]:
df.head()

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,2.058691,7645.670886,66.265823,3535.187526,450,True,1.0,0.0,0.0,Delay priority,1
0,2.124349,10317.088608,66.835443,4770.391441,450,True,1.0,0.0,0.0,Delay priority,2
0,2.334596,13882.632911,67.835443,6419.019525,450,True,1.0,0.0,0.0,Delay priority,3
0,0.438112,9883.493671,70.835443,-264.962121,450,True,0.0,1.0,0.0,Throughput priority,1
0,0.886684,12557.873418,71.835443,-268.702652,450,True,0.0,1.0,0.0,Throughput priority,2


In [12]:
df_initial_results['demand'] = df_initial_results.apply(lambda row: 
                                                        int(row['demand_ns']) if row['demand_ns'] == row['demand_ew']
                                                        else (int(row['demand_ns']), int(row['demand_ew']))
                                                        , axis=1)

df_initial_results['new_model'] = df_initial_results.apply(lambda row: row['alpha'] != 0, axis=1)

df_initial_results['model_type'] = df_initial_results.apply(lambda row: _model_type[
    (row['alpha'], row['beta'], row['gamma'])
], axis=1)

df_initial_results['epoch'] = 0

df_initial_results['delay'] = df_initial_results['delay'].apply(lambda x: 2*x)

In [13]:
df_initial_results = df_initial_results[[
    'runtime',
    'delay',
    'throughput',
    'objective_value',
    'demand',
    'new_model',
    'alpha',
    'beta',
    'gamma',
    'model_type',
    'epoch'
]]

In [14]:
df = df.rename(columns={'obj-value': 'objective_value'})

In [15]:
df_final = pd.concat([df_initial_results, df]).sort_values(by=['epoch', 'demand', 'model_type'])

In [16]:
save_df(df_final, 'milp-realtime-results')

In [17]:
df_final

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,165.902929,3069.088608,51.278481,709.539027,450,True,1.00,0.00,0.00,Delay priority,0
5,540.855196,3069.088608,51.278481,-666.881330,450,True,0.50,0.00,0.50,Delay-Flow priority,0
3,475.487772,3069.088608,51.278481,258.865157,450,True,0.50,0.50,0.00,Delay-Throughput priority,0
6,752.900086,3069.088608,51.278481,-503.438553,450,True,0.33,0.33,0.33,Equal priority,0
2,573.475265,3265.265823,52.278481,-2055.801688,450,False,0.00,0.00,1.00,Flow priority,0
7,370.309661,2313.000000,79.500000,79.500000,450,False,0.00,0.00,0.00,Parent model,0
1,268.785380,3500.000000,55.000000,-205.729167,450,False,0.00,1.00,0.00,Throughput priority,0
4,293.788045,3220.430380,54.000000,-1128.895162,450,False,0.00,0.50,0.50,Throughput-Flow priority,0
8,5.303937,8322.632646,60.000003,1902.982249,900,True,1.00,0.00,0.00,Delay priority,0
13,8.854174,8322.632911,60.000000,-717.970870,900,True,0.50,0.00,0.50,Delay-Flow priority,0
